# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - można zaimplementować w tym celu wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [14]:
from typing import Tuple, List

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove
import math
import copy
import random

Wielkość planszy

In [5]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [15]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game
        self.state = self.game.state


    def evaluate_position(self, player: Player, other: Player)->float:
        # heurystyka
        heuristic = 0
        for column_id in range(len(self.state.fields)):  # verticals
            for start_row_id in range(len(self.state.fields[column_id])):
                try:
                    if self.state.fields[column_id][start_row_id] == self.state.fields[column_id + 1][start_row_id] == player and (self.state.fields[column_id - 2][start_row_id] == self.state.fields[column_id - 1][start_row_id] == None or self.state.fields[column_id + 2][start_row_id] == self.state.fields[column_id + 3][start_row_id] == None):
                        heuristic += 10
                    if self.state.fields[column_id][start_row_id] == self.state.fields[column_id + 1][start_row_id] == self.state.fields[column_id + 2][start_row_id] == player and (self.state.fields[column_id - 1][start_row_id] == None or self.state.fields[column_id + 3][start_row_id] == None):
                        heuristic += 1000
                    if self.state.fields[column_id] == self.state.fields[column_id + 1][start_row_id] == self.state.fields[column_id + 2][start_row_id] == self.state.fields[column_id + 3][start_row_id] == player:
                        heuristic += 100000
                except IndexError:
                    pass
        for start_column_id in range(len(self.state.fields)):  # horizontals
            for row_id in range(len(self.state.fields[start_column_id])):
                try:
                    if self.state.fields[start_column_id][row_id] == self.state.fields[start_column_id][row_id + 1] == player and (self.state.fields[column_id][start_row_id - 2] == self.state.fields[column_id][start_row_id - 1] == None or self.state.fields[column_id][start_row_id + 2] == self.state.fields[column_id][start_row_id + 3] == None):
                        heuristic += 10
                    if self.state.fields[start_column_id][row_id] == self.state.fields[start_column_id][row_id + 1] == self.state.fields[start_column_id][row_id + 2] == player and (self.state.fields[column_id][start_row_id - 1] == None or self.state.fields[column_id][start_row_id + 3] == None):
                        heuristic += 1000
                    if self.state.fields[start_column_id][row_id] == self.state.fields[start_column_id][row_id + 1] == self.state.fields[start_column_id][row_id + 2] == self.state.fields[start_column_id][row_id + 3] == player:
                        heuristic += 100000
                except IndexError:
                    pass
        for start_column_id in range(len(self.state.fields)):  # diagonals positive
            for start_row_id in range(len(self.state.fields[start_column_id])):
                if start_row_id + 3 <= len(self.state.fields[start_column_id]) and start_column_id + 3 <= len(self.state.fields):
                    try:
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id + 1] == player and (self.state.fields[column_id - 2][start_row_id - 2] == self.state.fields[column_id - 1][start_row_id - 1] == None or self.state.fields[column_id + 2][start_row_id + 2] == self.state.fields[column_id + 3][start_row_id + 3] == None):
                            heuristic += 10
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id + 1] == self.state.fields[start_column_id + 2][start_row_id + 2] == player and (self.state.fields[column_id - 1][start_row_id - 1] == None or self.state.fields[column_id + 3][start_row_id + 3] == None):
                            heuristic += 1000
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id + 1] == self.state.fields[start_column_id + 2][start_row_id + 2] == self.state.fields[start_column_id + 3][start_row_id + 3] == player:
                            heuristic += 100000
                    except IndexError:
                        pass
        for start_column_id in range(len(self.state.fields)):  # diagonals negative
            for start_row_id in range(len(self.state.fields[start_column_id])):
                if start_row_id - 3 >= 0 and start_column_id + 3 <= len(self.state.fields):
                    try:
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id - 1] == player and (self.state.fields[column_id - 2][start_row_id + 2] == self.state.fields[column_id - 1][start_row_id + 1] == None or self.state.fields[column_id + 2][start_row_id - 2] == self.state.fields[column_id + 3][start_row_id - 3] == None):
                            heuristic += 10
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id - 1] == self.state.fields[start_column_id + 2][start_row_id - 2] == player and (self.state.fields[column_id - 1][start_row_id + 1] == None or self.state.fields[column_id + 3][start_row_id - 3] == None):
                            heuristic += 1000
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id - 1] == self.state.fields[start_column_id + 2][start_row_id - 2] == self.state.fields[start_column_id + 3][start_row_id - 3] == player:
                            heuristic += 100000
                    except IndexError:
                        pass
        for column_id in range(len(self.state.fields)):  # verticals
            for start_row_id in range(len(self.state.fields[column_id])):
                try:
                    if self.state.fields[column_id][start_row_id] == self.state.fields[column_id + 1][start_row_id] == other and (self.state.fields[column_id - 2][start_row_id] == self.state.fields[column_id - 1][start_row_id] == None or self.state.fields[column_id + 2][start_row_id] == self.state.fields[column_id + 3][start_row_id] == None):
                        heuristic -= 10
                    if self.state.fields[column_id][start_row_id] == self.state.fields[column_id + 1][start_row_id] == self.state.fields[column_id + 2][start_row_id] == other and (self.state.fields[column_id - 1][start_row_id] == None or self.state.fields[column_id + 3][start_row_id] == None):
                        heuristic -= 1000
                    if self.state.fields[column_id] == self.state.fields[column_id + 1][start_row_id] == self.state.fields[column_id + 2][start_row_id] == self.state.fields[column_id + 3][start_row_id] == other:
                        heuristic -= 100000
                except IndexError:
                    pass
        for start_column_id in range(len(self.state.fields)):  # horizontals
            for row_id in range(len(self.state.fields[start_column_id])):
                try:
                    if self.state.fields[start_column_id][row_id] == self.state.fields[start_column_id][row_id + 1] == other and (self.state.fields[column_id][start_row_id - 2] == self.state.fields[column_id][start_row_id - 1] == None or self.state.fields[column_id][start_row_id + 2] == self.state.fields[column_id][start_row_id + 3] == None):
                        heuristic -= 10
                    if self.state.fields[start_column_id][row_id] == self.state.fields[start_column_id][row_id + 1] == self.state.fields[start_column_id][row_id + 2] == other and (self.state.fields[column_id][start_row_id - 1] == None or self.state.fields[column_id][start_row_id + 3] == None):
                        heuristic -= 1000
                    if self.state.fields[start_column_id][row_id] == self.state.fields[start_column_id][row_id + 1] == self.state.fields[start_column_id][row_id + 2] == self.state.fields[start_column_id][row_id + 3] == other:
                        heuristic -= 100000
                except IndexError:
                    pass
        for start_column_id in range(len(self.state.fields)):  # diagonals positive
            for start_row_id in range(len(self.state.fields[start_column_id])):
                if start_row_id + 3 <= len(self.state.fields[start_column_id]) and start_column_id + 3 <= len(self.state.fields):
                    try:
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id + 1] == other and (self.state.fields[column_id - 2][start_row_id - 2] == self.state.fields[column_id - 1][start_row_id - 1] == None or self.state.fields[column_id + 2][start_row_id + 2] == self.state.fields[column_id + 3][start_row_id + 3] == None):
                            heuristic -= 10
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id + 1] == self.state.fields[start_column_id + 2][start_row_id + 2] == other and (self.state.fields[column_id - 1][start_row_id - 1] == None or self.state.fields[column_id + 3][start_row_id + 3] == None):
                            heuristic -= 1000
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id + 1] == self.state.fields[start_column_id + 2][start_row_id + 2] == self.state.fields[start_column_id + 3][start_row_id + 3] == other:
                            heuristic -= 100000
                    except IndexError:
                        pass
        for start_column_id in range(len(self.state.fields)):  # diagonals negative
            for start_row_id in range(len(self.state.fields[start_column_id])):
                if start_row_id - 3 >= 0 and start_column_id + 3 <= len(self.state.fields):
                    try:
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id - 1] == other and (self.state.fields[column_id - 2][start_row_id + 2] == self.state.fields[column_id - 1][start_row_id + 1] == None or self.state.fields[column_id + 2][start_row_id - 2] == self.state.fields[column_id + 3][start_row_id - 3] == None):
                            heuristic -= 10
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id - 1] == self.state.fields[start_column_id + 2][start_row_id - 2] == other and (self.state.fields[column_id - 1][start_row_id + 1] == None or self.state.fields[column_id + 3][start_row_id - 3] == None):
                            heuristic -= 1000
                        if self.state.fields[start_column_id][start_row_id] == self.state.fields[start_column_id + 1][start_row_id - 1] == self.state.fields[start_column_id + 2][start_row_id - 2] == self.state.fields[start_column_id + 3][start_row_id - 3] == other:
                            heuristic -= 100000
                    except IndexError:
                        pass
        return heuristic

    #def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
    # def minimax(self, node, depth, is_max) -> Tuple[int, float]:

    #     if is_max:
    #         return self.minimax(node, depth, -math.inf, math.inf, True)
    #     else:
    #         return self.minimax(node, depth, -math.inf, math.inf, False)

        
    def minimax(self, game, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        if depth == 0 or len(self._get_valid_locations()) == 0:
            # return None, self.evaluate_position(self.state._current_player, self.state._other_player)
            if is_maximizing_player:
                return None, self.evaluate_position(self.state._current_player, self.state._other_player)
            else:
                return None, -self.evaluate_position(self.state._current_player, self.state._other_player)
        if is_maximizing_player:
            best_score = -math.inf
            best_moves = []
            best_move = None
            for location in self._get_valid_locations():
                temp_game = copy.deepcopy(self.game)
                move = ConnectFourMove(location)
                temp_game.make_move(move)
                tmp_move, evaluation = self.minimax(temp_game, depth-1, alpha, beta, is_maximizing_player)
                if evaluation == best_score:
                    move_col = move.column
                    best_moves.append(move_col)
                if evaluation > best_score:
                    best_move = move.column
                    best_moves = [best_move]
                best_score = max(best_score, evaluation)
                alpha = max(alpha, best_score)
                if alpha >= beta:
                    break
            chosen_move = random.choice(best_moves)
            return chosen_move, best_score
        else:
            best_score = math.inf
            best_moves = []
            best_move = None
            for location in self._get_valid_locations():
                temp_game = copy.deepcopy(self.game)
                move = ConnectFourMove(location)
                temp_game.make_move(move)
                tmp_move, evaluation = self.minimax(temp_game, depth-1, alpha, beta, is_maximizing_player)
                if evaluation == best_score:
                    move_col = move.column
                    best_moves.append(move_col)
                if evaluation < best_score:
                    best_move = move.column
                    best_moves = [best_move]
                best_score = min(best_score, evaluation)
                beta = min(beta, best_score)
                if alpha >= beta:
                    break
            chosen_move = random.choice(best_moves)
            return chosen_move, best_score
        
        


    def get_best_move(self, game, depth, is_max):
        if is_max:
            move, value = self.minimax(game, depth, -math.inf, math.inf, True)
            best_move = ConnectFourMove(move)
        else:
            move, value = self.minimax(game, depth, -math.inf, math.inf, False)
            best_move = ConnectFourMove(move)
        return best_move
    
  

    def _get_valid_locations(self)->List[int]:
        valid_locations = []
        for column_id in range(len(self.state.fields)):
            if self._is_valid_move(column_id):
                # tmp_move = self.state.make_move(ConnectFourMove(column_id))
                valid_locations.append(column_id)
                # valid_locations.append(tmp_move)
        return valid_locations


    def _is_valid_move(self, col_index:int)->bool:
        for row_val in self.state.fields[col_index]:
            if row_val is None:
                return True
        return False




Rozgrywka

In [7]:
def main(depth_a, depth_b):
    p1 = Player("a")
    p2 = Player("b")
    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
    print(game)
    strategy = MinMaxSolver(game)
    while game.is_finished() is False:
        current_player = game.state.get_current_player()
        if current_player.char == game.second_player.char:
            game.make_move(strategy.get_best_move(game, depth_b, False))
        if current_player.char == game.first_player.char:
            game.make_move(strategy.get_best_move(game, depth_a, True))
    
            
        print(game)
    winner = game.get_winner()   
    print(f' THE WINNER IS {winner.char}')

In [16]:
"""Rozgrywka"""
main(3, 3)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][a][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][a][b][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][a][b][a]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][a][b][a]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][b][ ][a][b][a]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

Przy takich samych głębokościach przeszukiwań równych 3 tym razem wygrał gracz maksymalizujący

In [18]:
# różne głębokości
main(4, 3)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][a][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][b][a][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][a][ ][ ][b][a][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][ ][ ]
[ ][a][ ][ ][b][a][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][ ][ ][ ][a][ ]
[ ][a][ ][ ][b][a][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

W tej rozgrywce wygrał gracz o większej głębokości przeszukiwań. Dalej sprawdzę, czy taki sam efekt osiągnę, gdy to gracz minimalizujący będzie miał większą głębokość przeszukiwań.

In [11]:
main(3, 4)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][a][b][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][ ][ ][ ]
[ ][ ][a][a][b][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][ ][ ][ ]
[ ][ ][a][a][b][ ][a]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

W tym przypadku również wygrał gracz o większej głębokości przeszukiwań. Nie dzieje się tak jednak w każdej rozgrywce.

In [12]:
# głębokość 5 dla obu graczy
main(5, 5)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][a]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][ ][ ][a]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][a]
[ ][ ][b][ ][ ][ ][a]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][a]
[ ][ ][b][b][ ][ ][a]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][a]
[ ][ ][b][b][a][ ][a]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

Tak jak w większości przeprowadzonych przeze mnie doświadczeń, wygrał gracz, który rozpoczyna ruch jako pierwszy. Dla głębokości 5 algorytm wykonuje się kilka minut, ze względu na obecną w nim rekurencję. Sprawdzę teraz, jaki wynik otrzymam dla głębokości przeszukiwań 1.

In [13]:
# głębokość 1 dla obu graczy
main(1, 1)

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][a][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][ ][a][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][b][ ][a][a][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][b][ ][a][a][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][b][b][a][a][a][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

Znów zwycięzcą okazał się gracz rozpoczynający. 

"""WNIOSKI"""
Nie można pominąć faktu, że algorytm nie działa idealnie ze względu na funkcję heurystyczną, która jest trudna do określenia. Moja skonstruowana jest tak, że za każdą "dwójkę" w szeregu z dwoma wolnymi sąsiadującymi polami funkcja zwraca 1, za każdą trójkę z sąsiadującym wolnym polem zwraca 1000, a za czwórkę - 100000. Analogicznie funkcja odejmuje te same wartości dla "streaków" u przeciwnika. Wynika z niej, że gracz ma taką samą nagrodę za dążenie do własnej przewagi, co karę za nieblokowanie przeciwnika. Poza tym, ruchy o tej samej wartości są wybierane losowo, żeby otrzymywać różne rozgrywki dla tych samych głębokości. Dla zbyt dużej głębokości, algorytm przestawał działać optymalnie - pomijał miejsca wygrane i nie blokuje przeciwnika. W większości przeprowadzonych przeze mnie rozgrywek wygrywał gracz rozpoczynający, co zgadza się z informacjami na temat rozwiązania gry, które przeczytałam.